In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Load the Dataset**

In [ ]:
df=pd.read_csv('/kaggle/input/churn-modelling/Churn_Modelling.csv')

In [ ]:
df.head()

In [ ]:
df.shape

### **Missing Values**

In [ ]:
df.isnull().sum().sum()

There is no missing value

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16,9))
sns.heatmap(df.corr(),annot=True,cmap='RdYlGn')

### **Spliting of Data**

In [ ]:
X=df.iloc[:,3:13]
Y=df.iloc[:,13]

### **Conversion of Categorical to Numerical**

In [ ]:
geography=pd.get_dummies(X['Geography'],drop_first=True)#drop_first=True skips first column

In [ ]:
geography

In [ ]:
gender=pd.get_dummies(X['Gender'],drop_first=True)
gender

In [ ]:
X=X.drop(['Geography','Gender'],axis=1)
X.head()

In [ ]:
X=pd.concat([X,geography,gender],axis=1)
X.head()

### **Hyperparameter Tuning**

In [ ]:
params={
    'learning_rate':[0.05,0.1,0.15,0.2,0.25,0.3],
    'max_depth':[3,4,5,6,8,10,12,15],
    'min_child_weight':[1,3,5,7],
    'gamma':[0.0,0.1,0.2,0.3,0.4],
    'colsample_bytree':[0.1,0.4,0.5,0.7]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier

In [ ]:
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now()
        return start_time
    elif start_time:
        thour,temp_sec=divmod((datetime.now()-start_time).total_seconds(),3600)
        tmin,tsec=divmod(temp_sec,60)
        print('\n Time taken: %i hours, %i mins and %s secs'%(thour,tmin,round(tsec,2)))

In [ ]:
clf=XGBClassifier()

In [ ]:
random_search=RandomizedSearchCV(clf,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=1)

In [ ]:
start_time=timer(None)
random_search.fit(X,Y)
timer(start_time)

In [ ]:
random_search.best_estimator_

In [ ]:
random_search.best_params_

In [ ]:
clf=XGBClassifier(min_child_weight= 5,
 max_depth= 5,
 learning_rate= 0.15,
 gamma= 0.0,
 colsample_bytree= 0.5)

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(clf,X,Y,cv=10)

In [ ]:
score

In [ ]:
score.mean()

# **Cross Validation**

**K-Fold**

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(clf,X,Y,cv=10)

In [ ]:
score

In [ ]:
score.mean()

**Stratified K fold Cross Validation**

In [ ]:
X.shape,Y.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
accuracy=[]
skf=StratifiedKFold(n_splits=10,random_state=None)
skf.split(X,Y)
for train_index,test_index in skf.split(X,Y):
    print("Train: ",train_index,"Validation: ",test_index)
    x1_train,x1_test=X.iloc[train_index],X.iloc[test_index]
    y1_train,y1_test=Y.iloc[train_index],Y.iloc[test_index]
    clf.fit(x1_train,y1_train)
    predictions=clf.predict(x1_test)
    score=accuracy_score(predictions,y1_test)
    accuracy.append(score)
print(accuracy)

In [ ]:
# skf.split()
import numpy as np
np.array(accuracy).mean()